In [2]:
import argparse
import yaml
from os.path import exists, join
import xarray as xr
import numpy as np
from glob import glob
from dask.distributed import Client, LocalCluster, wait
import traceback
from os import makedirs
import cftime

In [3]:
# Load the NetCDF file
file_path = '/glade/work/wchuang/mlmicrophysics/e3sm300_mlmicro11.1_tau_train/e3sm300_mlmicro11.1_tau_train.eam.h1.2001-07-04-00000.nc'  # Replace with your NetCDF file path
# dataset = nc.Dataset(file_path)
ds = xr.open_dataset(file_path)

In [4]:
ds

<xarray.Dataset>
Dimensions:                 (ncol: 21600, lev: 80, ilev: 81, P3_input_dim: 16,
                             P3_output_dim: 32, cosp_prs: 7, nbnd: 2,
                             cosp_tau: 7, cosp_scol: 10, cosp_ht: 40,
                             cosp_temp: 40, cosp_sr: 15, cosp_sza: 5,
                             cosp_htmisr: 16, cosp_tau_modis: 7,
                             cosp_reffice: 6, cosp_reffliq: 6, time: 32)
Coordinates: (12/16)
  * lev                     (lev) float64 0.1236 0.1819 0.2674 ... 993.8 998.5
  * ilev                    (ilev) float64 0.1 0.1472 0.2166 ... 997.0 1e+03
  * P3_input_dim            (P3_input_dim) int32 1 2 3 4 5 6 ... 12 13 14 15 16
  * P3_output_dim           (P3_output_dim) int32 1 2 3 4 5 6 ... 28 29 30 31 32
  * cosp_prs                (cosp_prs) float64 9e+04 7.4e+04 ... 2.45e+04 9e+03
  * cosp_tau                (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 100.0
    ...                      ...
  * cosp_sza                (cosp_sza) float64 0.0 20.0 40.0 60.0 80.0
  * cosp_htmisr             (cosp_htmisr) float64 0.0 250.0 ... 1.6e+04 1.8e+04
  * cosp_tau_modis          (cosp_tau_modis) float64 0.15 0.8 ... 41.5 100.0
  * cosp_reffice            (cosp_reffice) float64 5e-06 1.5e-05 ... 7.5e-05
  * cosp_reffliq            (cosp_reffliq) float64 4e-06 9e-06 ... 2.5e-05
  * time                    (time) object 2001-07-04 00:00:00 ... 2001-08-02 ...
Dimensions without coordinates: ncol, nbnd
Data variables: (12/77)
    lat                     (ncol) float64 ...
    lon                     (ncol) float64 ...
    area                    (ncol) float64 ...
    hyam                    (lev) float64 ...
    hybm                    (lev) float64 ...
    P0                      float64 ...
    ...                      ...
    RELHUM                  (time, lev, ncol) float32 ...
    RHO_CLUBB               (time, ilev, ncol) float32 ...
    SWCF                    (time, ncol) float32 ...
    T                       (time, lev, ncol) float32 ...
    TGCLDIWP                (time, ncol) float32 ...
    TGCLDLWP                (time, ncol) float32 ...
Attributes: (12/19)
    ne:                30
    fv_nphys:          2
    title:             EAM History file information
    source:            E3SM Atmosphere Model
    source_id:         a67353c6a2
    product:           model-output
    ...                ...
    institution_id:    E3SM-Project
    institution:       LLNL (Lawrence Livermore National Laboratory, Livermor...
    contact:           e3sm-data-support@llnl.gov
    initial_file:      /global/cfs/cdirs/e3sm/inputdata/atm/cam/inic/homme/ea...
    topography_file:   /global/cfs/cdirs/e3sm/inputdata/atm/cam/topo/USGS-gto...
    time_period_freq:  hour_23

In [4]:
ds.data_vars.keys()

KeysView(Data variables:
    lat                     (ncol) float64 ...
    lon                     (ncol) float64 ...
    area                    (ncol) float64 ...
    hyam                    (lev) float64 ...
    hybm                    (lev) float64 ...
    P0                      float64 ...
    hyai                    (ilev) float64 ...
    hybi                    (ilev) float64 ...
    cosp_prs_bnds           (cosp_prs, nbnd) float64 ...
    cosp_tau_bnds           (cosp_tau, nbnd) float64 ...
    cosp_ht_bnds            (cosp_ht, nbnd) float64 ...
    cosp_temp_bnds          (cosp_temp, nbnd) float64 ...
    cosp_sr_bnds            (cosp_sr, nbnd) float64 ...
    cosp_htmisr_bnds        (cosp_htmisr, nbnd) float64 ...
    cosp_tau_modis_bnds     (cosp_tau_modis, nbnd) float64 ...
    cosp_reffice_bnds       (cosp_reffice, nbnd) float64 ...
    cosp_reffliq_bnds       (cosp_reffliq, nbnd) float64 ...
    date                    (time) int32 ...
    datesec                 (time)

In [5]:
out_variables = ["T", "RHO_CLUBB", 
                "CLOUD", "FREQR",
                "P3_qc_in_TAU", "P3_nc_in_TAU", "P3_qr_in_TAU", "P3_nr_in_TAU", 
                "P3_qc_out_TAU", "P3_nc_out_TAU", "P3_qr_out_TAU", "P3_nr_out_TAU",
                "P3_qctend_TAU_raw", "P3_nctend_TAU_raw", "P3_qrtend_TAU_raw", "P3_nrtend_TAU_raw", 
                "P3_mu_c", "P3_lamc", "P3_lamr", "P3_nr"]

In [6]:
ds_important = ds[out_variables]

In [8]:
ds_important.to_netcdf('/glade/work/wchuang/mlmicrophysics/e3sm300_mlmicro11.1_tau_train/e3sm300_mlmicro11.1_tau_train.eam.h1.2001-07-04-00000_filter01.nc')

#### start here for filtered data

In [9]:
ds_important = xr.open_dataset('/glade/work/wchuang/mlmicrophysics/e3sm300_mlmicro11.1_tau_train/e3sm300_mlmicro11.1_tau_train.eam.h1.2001-07-04-00000_filter01.nc')

In [6]:
times = ds["time"]
time_hours = int((times[0] - times[0]).item() / 3600 / 1e9)
print(time_hours)
time_df = ds[out_variables].sel(**{"time": times[0]}).to_dataframe()

0


In [7]:
valid = np.zeros(time_df.shape[0], dtype=bool)
valid = time_df[subset_variable] >= subset_threshold

T  RHO_CLUBB  CLOUD  FREQR  \
lev        ncol  ilev                                               
0.123611   0     0.100000     240.502243   0.000179    0.0    0.0   
                 0.147223     240.502243   0.000255    0.0    0.0   
                 0.216607     240.502243   0.000377    0.0    0.0   
                 0.318250     240.502243   0.000539    0.0    0.0   
                 0.466509     240.502243   0.000758    0.0    0.0   
...                                  ...        ...    ...    ...   
998.496439 21599 972.738495   296.194366   1.146617    0.0    1.0   
                 981.889633   296.194366   1.154427    0.0    1.0   
                 990.521034   296.194366   1.160773    0.0    1.0   
                 996.992879   296.194366   1.164676    0.0    1.0   
                 1000.000000  296.194366   1.166426    0.0    1.0   

                              P3_qc_in_TAU  P3_nc_in_TAU  P3_qr_in_TAU  \
lev        ncol  ilev                                                    
0.123611   0     0.100000              0.0           0.0  0.000000e+00   
                 0.147223              0.0           0.0  0.000000e+00   
                 0.216607              0.0           0.0  0.000000e+00   
                 0.318250              0.0           0.0  0.000000e+00   
                 0.466509              0.0           0.0  0.000000e+00   
...                                    ...           ...           ...   
998.496439 21599 972.738495            0.0           0.0  3.013922e-07   
                 981.889633            0.0           0.0  3.013922e-07   
                 990.521034            0.0           0.0  3.013922e-07   
                 996.992879            0.0           0.0  3.013922e-07   
                 1000.000000           0.0           0.0  3.013922e-07   

                              P3_nr_in_TAU  P3_qc_out_TAU  P3_nc_out_TAU  ...  \
lev        ncol  ilev                                                     ...   
0.123611   0     0.100000         0.000000            0.0            0.0  ...   
                 0.147223         0.000000            0.0            0.0  ...   
                 0.216607         0.000000            0.0            0.0  ...   
                 0.318250         0.000000            0.0            0.0  ...   
                 0.466509         0.000000            0.0            0.0  ...   
...                                    ...            ...            ...  ...   
998.496439 21599 972.738495       0.804598            0.0            0.0  ...   
                 981.889633       0.804598            0.0            0.0  ...   
                 990.521034       0.804598            0.0            0.0  ...   
                 996.992879       0.804598            0.0            0.0  ...   
                 1000.000000      0.804598            0.0            0.0  ...   

                              P3_nr_out_TAU  P3_qctend_TAU_raw  \
lev        ncol  ilev                                            
0.123611   0     0.100000          0.000000                0.0   
                 0.147223          0.000000                0.0   
                 0.216607          0.000000                0.0   
                 0.318250          0.000000                0.0   
                 0.466509          0.000000                0.0   
...                                     ...                ...   
998.496439 21599 972.738495        0.729969                0.0   
                 981.889633        0.729969                0.0   
                 990.521034        0.729969                0.0   
                 996.992879        0.729969                0.0   
                 1000.000000       0.729969                0.0   

                              P3_nctend_TAU_raw  P3_qrtend_TAU_raw  \
lev        ncol  ilev                                                
0.123611   0     0.100000                   0.0                0.0   
                 0.147223                   0.0        

In [147]:
ds1

<xarray.Dataset>
Dimensions:                 (ncol: 21600, lev: 80, ilev: 81, P3_input_dim: 16,
                             P3_output_dim: 32, cosp_prs: 7, nbnd: 2,
                             cosp_tau: 7, cosp_scol: 10, cosp_ht: 40,
                             cosp_temp: 40, cosp_sr: 15, cosp_sza: 5,
                             cosp_htmisr: 16, cosp_tau_modis: 7,
                             cosp_reffice: 6, cosp_reffliq: 6, time: 32)
Coordinates: (12/18)
    lat                     (ncol) float64 -34.91 -35.58 -33.51 ... 36.66 35.97
    lon                     (ncol) float64 315.7 317.2 315.7 ... 136.5 135.0
  * lev                     (lev) float64 0.1236 0.1819 0.2674 ... 993.8 998.5
  * ilev                    (ilev) float64 0.1 0.1472 0.2166 ... 997.0 1e+03
  * P3_input_dim            (P3_input_dim) int32 1 2 3 4 5 6 ... 12 13 14 15 16
  * P3_output_dim           (P3_output_dim) int32 1 2 3 4 5 6 ... 28 29 30 31 32
    ...                      ...
  * cosp_sza                (cosp_sza) float64 0.0 20.0 40.0 60.0 80.0
  * cosp_htmisr             (cosp_htmisr) float64 0.0 250.0 ... 1.6e+04 1.8e+04
  * cosp_tau_modis          (cosp_tau_modis) float64 0.15 0.8 ... 41.5 100.0
  * cosp_reffice            (cosp_reffice) float64 5e-06 1.5e-05 ... 7.5e-05
  * cosp_reffliq            (cosp_reffliq) float64 4e-06 9e-06 ... 2.5e-05
  * time                    (time) object 2001-07-04 00:00:00 ... 2001-08-02 ...
Dimensions without coordinates: ncol, nbnd
Data variables: (12/75)
    area                    (ncol) float64 ...
    hyam                    (lev) float64 ...
    hybm                    (lev) float64 ...
    P0                      float64 ...
    hyai                    (ilev) float64 ...
    hybi                    (ilev) float64 ...
    ...                      ...
    RELHUM                  (time, lev, ncol) float32 ...
    RHO_CLUBB               (time, ilev, ncol) float32 ...
    SWCF                    (time, ncol) float32 ...
    T                       (time, lev, ncol) float32 ...
    TGCLDIWP                (time, ncol) float32 ...
    TGCLDLWP                (time, ncol) float32 ...
Attributes: (12/19)
    ne:                30
    fv_nphys:          2
    title:             EAM History file information
    source:            E3SM Atmosphere Model
    source_id:         a67353c6a2
    product:           model-output
    ...                ...
    institution_id:    E3SM-Project
    institution:       LLNL (Lawrence Livermore National Laboratory, Livermor...
    contact:           e3sm-data-support@llnl.gov
    initial_file:      /global/cfs/cdirs/e3sm/inputdata/atm/cam/inic/homme/ea...
    topography_file:   /global/cfs/cdirs/e3sm/inputdata/atm/cam/topo/USGS-gto...
    time_period_freq:  hour_23

In [71]:
ds1.coords

Coordinates:
    lat             (ncol) float64 -34.91 -35.58 -33.51 ... 36.66 36.66 35.97
    lon             (ncol) float64 315.7 317.2 315.7 317.2 ... 133.5 136.5 135.0
  * lev             (lev) float64 0.1236 0.1819 0.2674 ... 986.2 993.8 998.5
  * ilev            (ilev) float64 0.1 0.1472 0.2166 ... 990.5 997.0 1e+03
  * P3_input_dim    (P3_input_dim) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16
  * P3_output_dim   (P3_output_dim) int32 1 2 3 4 5 6 7 ... 26 27 28 29 30 31 32
  * cosp_prs        (cosp_prs) float64 9e+04 7.4e+04 6.2e+04 ... 2.45e+04 9e+03
  * cosp_tau        (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 100.0
  * cosp_scol       (cosp_scol) int32 1 2 3 4 5 6 7 8 9 10
  * cosp_ht         (cosp_ht) float64 1.896e+04 1.848e+04 ... 720.0 240.0
  * cosp_temp       (cosp_temp) float64 -91.5 -88.5 -85.5 ... 19.5 22.5 25.5
  * cosp_sr         (cosp_sr) float64 -0.495 0.605 2.1 4.0 ... 55.0 70.0 539.5
  * cosp_sza        (cosp_sza) float64 0.0 20.0 40.0 60.0 80.0
  * cosp_h

In [73]:
# 1. Print basic information about the NetCDF file
def print_basic_info(dataset):
    print("File format:", dataset.file_format)
    print("\nGlobal Attributes:")
    for attr in dataset.ncattrs():
        print(f"{attr}: {dataset.getncattr(attr)}")
    
    print("\nDimensions:")
    for dim in dataset.dimensions.values():
        print(dim)
        
    print("\nVariables:")
    for var in dataset.variables.values():
        print(var)
        
print_basic_info(dataset)

File format: NETCDF3_64BIT_OFFSET

Global Attributes:
ne: 30
fv_nphys: 2
title: EAM History file information
source: E3SM Atmosphere Model
source_id: a67353c6a2
product: model-output
realm: atmos
case: e3sm300_mlmicro11.1_tau_train
username: agett
hostname: pm-cpu
git_version: a67353c6a2
history: created on 11/06/24 09:05:33
Conventions: CF-1.7
institution_id: E3SM-Project
institution: LLNL (Lawrence Livermore National Laboratory, Livermore, CA 94550, USA); ANL (Argonne National Laboratory, Argonne, IL 60439, USA); BNL (Brookhaven National Laboratory, Upton, NY 11973, USA); LANL (Los Alamos National Laboratory, Los Alamos, NM 87545, USA); LBNL (Lawrence Berkeley National Laboratory, Berkeley, CA 94720, USA); ORNL (Oak Ridge National Laboratory, Oak Ridge, TN 37831, USA); PNNL (Pacific Northwest National Laboratory, Richland, WA 99352, USA); SNL (Sandia National Laboratories, Albuquerque, NM 87185, USA). Mailing address: LLNL Climate Program, c/o David C. Bader, Principal Investigator, 

In [14]:
### 2. List all variables and their dimensions
def list_variables(dataset):
    for var_name, variable in dataset.variables.items():
        print(f"\nVariable: {var_name}")
        print("Dimensions:", variable.dimensions)
        print("Shape:", variable.shape)
        print("Attributes:")
        for attr in variable.ncattrs():
            print(f"  {attr}: {variable.getncattr(attr)}")
            
list_variables(dataset)


Variable: lat
Dimensions: ('ncol',)
Shape: (21600,)
Attributes:
  long_name: latitude
  units: degrees_north

Variable: lon
Dimensions: ('ncol',)
Shape: (21600,)
Attributes:
  long_name: longitude
  units: degrees_east

Variable: area
Dimensions: ('ncol',)
Shape: (21600,)
Attributes:
  long_name: physics grid areas

Variable: lev
Dimensions: ('lev',)
Shape: (80,)
Attributes:
  long_name: hybrid level at midpoints (1000*(A+B))
  units: hPa
  positive: down
  standard_name: atmosphere_hybrid_sigma_pressure_coordinate
  formula_terms: a: hyam b: hybm p0: P0 ps: PS

Variable: hyam
Dimensions: ('lev',)
Shape: (80,)
Attributes:
  long_name: hybrid A coefficient at layer midpoints

Variable: hybm
Dimensions: ('lev',)
Shape: (80,)
Attributes:
  long_name: hybrid B coefficient at layer midpoints

Variable: P0
Dimensions: ()
Shape: ()
Attributes:
  long_name: reference pressure
  units: Pa

Variable: ilev
Dimensions: ('ilev',)
Shape: (81,)
Attributes:
  long_name: hybrid level at interfaces (10

### Look at how filtering affects data

In [10]:
ds_important

<xarray.Dataset>
Dimensions:            (time: 32, lev: 80, ncol: 21600, ilev: 81)
Coordinates:
  * lev                (lev) float64 0.1236 0.1819 0.2674 ... 986.2 993.8 998.5
  * ilev               (ilev) float64 0.1 0.1472 0.2166 ... 990.5 997.0 1e+03
  * time               (time) object 2001-07-04 00:00:00 ... 2001-08-02 17:00:00
Dimensions without coordinates: ncol
Data variables: (12/20)
    T                  (time, lev, ncol) float32 ...
    RHO_CLUBB          (time, ilev, ncol) float32 ...
    CLOUD              (time, lev, ncol) float32 ...
    FREQR              (time, lev, ncol) float32 ...
    P3_qc_in_TAU       (time, lev, ncol) float32 ...
    P3_nc_in_TAU       (time, lev, ncol) float32 ...
    ...                 ...
    P3_qrtend_TAU_raw  (time, lev, ncol) float32 ...
    P3_nrtend_TAU_raw  (time, lev, ncol) float32 ...
    P3_mu_c            (time, lev, ncol) float32 ...
    P3_lamc            (time, lev, ncol) float32 ...
    P3_lamr            (time, lev, ncol) float32 ...
    P3_nr              (time, lev, ncol) float32 ...
Attributes: (12/19)
    ne:                30
    fv_nphys:          2
    title:             EAM History file information
    source:            E3SM Atmosphere Model
    source_id:         a67353c6a2
    product:           model-output
    ...                ...
    institution_id:    E3SM-Project
    institution:       LLNL (Lawrence Livermore National Laboratory, Livermor...
    contact:           e3sm-data-support@llnl.gov
    initial_file:      /global/cfs/cdirs/e3sm/inputdata/atm/cam/inic/homme/ea...
    topography_file:   /global/cfs/cdirs/e3sm/inputdata/atm/cam/topo/USGS-gto...
    time_period_freq:  hour_23

In [4]:
ds_important = ds_important.drop_vars(['P0', 'ndbase', 'nsbase', 'nbdate', 'nbsec', 'mdt'])

In [16]:
filter_mask = (ds_important.data_vars['CLOUD'] > 1e-2) # & (ds_important.data_vars['P3_qc_in_TAU'] > 1e-6)# & (ds_important.data_vars['P3_qctend_TAU_raw'] < 0)

In [17]:
sum(sum(sum(filter_mask)))

<xarray.DataArray 'CLOUD' ()>
array(8061096)

In [ ]:
ds2 = ds.where(ds['CLOUD'] != 0, drop=True)

In [7]:
# 3. Summary statistics for each variable
def summarize_variables(ds):
    print("\nSummary Statistics:")
    for var in ds.data_vars:
        print(f"\nVariable: {var}")
        print(ds[var].to_series().describe())

summarize_variables(ds3)


Summary Statistics:

Variable: lat
count    2.160000e+04
mean    -2.694799e-15
std      3.889868e+01
min     -8.893940e+01
25%     -3.075809e+01
50%     -1.554312e-15
75%      3.075809e+01
max      8.893940e+01
Name: lat, dtype: float64

Variable: lon
count    21600.000000
mean       180.000000
std        103.828999
min          0.750000
25%         90.375000
50%        180.000000
75%        269.625000
max        359.250000
Name: lon, dtype: float64

Variable: area
count    21600.000000
mean         0.000582
std          0.000052
min          0.000491
25%          0.000538
50%          0.000575
75%          0.000623
max          0.000685
Name: area, dtype: float64

Variable: hyam
count    80.000000
mean      0.061078
std       0.057099
min       0.000124
25%       0.011420
50%       0.041467
75%       0.104481
max       0.178834
Name: hyam, dtype: float64

Variable: hybm
count    80.000000
mean      0.286002
std       0.377347
min       0.000000
25%       0.000000
50%       0.000000
7

In [ ]:
# 4. Basic plot of a variable (replace 'variable_name' with an actual variable name from your dataset)
def plot_variable(ds, var_name):
    if var_name not in ds.variables:
        print(f"{var_name} not found in dataset.")
        return

    data = ds[var_name]
    if data.ndim == 2:  # 2D plot for simple spatial data
        data.plot()
        plt.title(f"{var_name} (2D Plot)")
    elif data.ndim == 3:  # 3D plot for time-series spatial data
        time_slice = data.isel(time=0)  # Select the first time slice
        time_slice.plot()
        plt.title(f"{var_name} at time=0")
    else:
        print(f"Cannot plot variable '{var_name}' with dimensions {data.ndim}")
    plt.show()

plot_variable(ds, 'CLDLIQ')  # Replace 'variable_name' with an actual variable name

In [ ]:
# 5. Close the dataset when done
dataset.close()
ds.close()

# Issues

Dealing with memory issue: 
<Client: 'tcp://127.0.0.1:39885' processes=36 threads=72, memory=0.98 TiB>
2024-11-22 22:18:04,087 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 19.66 GiB -- Worker memory limit: 27.78 GiB

2024-11-22 22:18:04,110 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 19.66 GiB -- Worker memory limit: 27.78 GiB

2024-11-22 22:18:04,159 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 19.75 GiB -- Worker memory limit: 27.78 GiB

2024-11-22 22:18:04,263 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 19.47 GiB -- Worker memory limit: 27.78 GiB

2024-11-22 22:18:04,300 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 19.45 GiB -- Worker memory limit: 27.78 GiB

In [2]:
import argparse
import yaml
from os.path import exists, join
import xarray as xr
import numpy as np
from mlmicrophysics.data import split_staggered_variable
from mlmicrophysics.data import load_cam_output, unstagger_vertical, convert_to_dataframe
from mlmicrophysics.data import calc_pressure_field, calc_temperature, add_index_coords
# from glob import glob
# from dask.distributed import Client, LocalCluster, wait
import traceback
from os import makedirs
from cftime import DatetimeNoLeap

In [3]:
out_variables = ["T", "RHO_CLUBB",
                 "CLOUD", "FREQR",
                 "P3_qc_in_TAU", "P3_nc_in_TAU", "P3_qr_in_TAU", "P3_nr_in_TAU",
                 "P3_qc_out_TAU", "P3_nc_out_TAU", "P3_qr_out_TAU", "P3_nr_out_TAU",
                 "P3_qctend_TAU_raw", "P3_nctend_TAU_raw", "P3_qrtend_TAU_raw", "P3_nrtend_TAU_raw",
                 "P3_mu_c", "P3_lamc", "P3_lamr", "P3_nr"]
subset_variable = ["P3_qc_in_TAU"]
subset_threshold = [1.0e-8]
out_path = "/glade/derecho/scratch/wchuang/mlmicrophysics/e3sm_ml_tau1/20241115/"
out_start = "e3sm_mp_data_tau1"
out_format = "parquet"

In [4]:
model_ds = xr.open_dataset('/glade/work/wchuang/mlmicrophysics/e3sm300_mlmicro11.1_tau_train/e3sm300_mlmicro11.1_tau_train.eam.h1.2001-04-03-00000.nc')

In [7]:
times = model_ds["time"]
time_hours = int((times[0] - times[0]).item() / 3600 / 1e9)
print(time_hours)
time_df = model_ds[out_variables].sel(**{"time": times[0]}).to_dataframe()

if type(subset_variable) == list:
    valid = np.zeros(time_df.shape[0], dtype=bool)
    for s, sv in enumerate(subset_variable):
        valid[time_df[sv] >= subset_threshold[s]] = True
else:
    valid = time_df[subset_variable] >= subset_threshold
time_sub_df = time_df.loc[valid].reset_index()


0


In [13]:
time_df["time"]

lev         ncol   ilev       
0.123611    0      0.100000       2001-04-03 00:00:00
                   0.147223       2001-04-03 00:00:00
                   0.216607       2001-04-03 00:00:00
                   0.318250       2001-04-03 00:00:00
                   0.466509       2001-04-03 00:00:00
                                         ...         
998.496439  21599  972.738495     2001-04-03 00:00:00
                   981.889633     2001-04-03 00:00:00
                   990.521034     2001-04-03 00:00:00
                   996.992879     2001-04-03 00:00:00
                   1000.000000    2001-04-03 00:00:00
Name: time, Length: 139968000, dtype: object

In [8]:
time_sub_df

,lev,ncol,ilev,T,RHO_CLUBB,CLOUD,FREQR,P3_qc_in_TAU,P3_nc_in_TAU,P3_qr_in_TAU,...,P3_nr_out_TAU,P3_qctend_TAU_raw,P3_nctend_TAU_raw,P3_qrtend_TAU_raw,P3_nrtend_TAU_raw,P3_mu_c,P3_lamc,P3_lamr,P3_nr,time
0,104.662866,963,0.100000,194.261658,0.000168,0.004069,0.0,2.344754e-06,56802.792969,0.000000e+00,...,33.589260,-3.089585e-09,-188.684235,3.089585e-09,0.111964,12.576265,339406.625,0.000000,1.000000,2001-04-03 00:00:00
1,104.662866,963,0.147223,194.261658,0.000236,0.004069,0.0,2.344754e-06,56802.792969,0.000000e+00,...,33.589260,-3.089585e-09,-188.684235,3.089585e-09,0.111964,12.576265,339406.625,0.000000,1.000000,2001-04-03 00:00:00
2,104.662866,963,0.216607,194.261658,0.000353,0.004069,0.0,2.344754e-06,56802.792969,0.000000e+00,...,33.589260,-3.089585e-09,-188.684235,3.089585e-09,0.111964,12.576265,339406.625,0.000000,1.000000,2001-04-03 00:00:00
3,104.662866,963,0.318250,194.261658,0.000516,0.004069,0.0,2.344754e-06,56802.792969,0.000000e+00,...,33.589260,-3.089585e-09,-188.684235,3.089585e-09,0.111964,12.576265,339406.625,0.000000,1.000000,2001-04-03 00:00:00
4,104.662866,963,0.466509,194.261658,0.000754,0.004069,0.0,2.344754e-06,56802.792969,0.000000e+00,...,33.589260,-3.089585e-09,-188.684235,3.089585e-09,0.111964,12.576265,339406.625,0.000000,1.000000,2001-04-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9344722,998.496439,21220,972.738495,268.935120,1.191666,0.606508,0.0,6.512762e-08,471541.812500,8.398183e-11,...,0.006002,-4.793903e-13,-618.492615,4.793903e-13,-0.000003,12.543756,2264167.000,6374.415527,44.136398,2001-04-03 00:00:00
9344723,998.496439,21220,981.889633,268.935120,1.199477,0.606508,0.0,6.512762e-08,471541.812500,8.398183e-11,...,0.006002,-4.793903e-13,-618.492615,4.793903e-13,-0.000003,12.543756,2264167.000,6374.415527,44.136398,2001-04-03 00:00:00
9344724,998.496439,21220,990.521034,268.935120,1.206066,0.606508,0.0,6.512762e-08,471541.812500,8.398183e-11,...,0.006002,-4.793903e-13,-618.492615,4.793903e-13,-0.000003,12.543756,2264167.000,6374.415527,44.136398,2001-04-03 00:00:00
9344725,998.496439,21220,996.992879,268.935120,1.210174,0.606508,0.0,6.512762e-08,471541.812500,8.398183e-11,...,0.006002,-4.793903e-13,-618.492615,4.793903e-13,-0.000003,12.543756,2264167.000,6374.415527,44.136398,2001-04-03 00:00:00
